In [3]:
using DMUStudent.HW3: HW3, DenseGridWorld, visualize_tree
using POMDPs: actions, @gen, isterminal, discount, statetype, actiontype, simulate, states, initialstate
using D3Trees: inchrome
using StaticArrays: SA
using Statistics: mean, std, mean

In [4]:
##############
# Instructions
##############
#=

This starter code is here to show examples of how to use the HW3 code that you
can copy and paste into your homework code if you wish. It is not meant to be a
fill-in-the blank skeleton code, so the structure of your final submission may
differ from this considerably.

Please make sure to update DMUStudent to gain access to the HW3 module.

=#

############
# Question 2
############


mdp = HW3.DenseGridWorld(seed = 3)
a = actions(mdp)
s = states(mdp)
policy_function = rand(actions(mdp))
start_state = SA[19, 19]

function rollout(mdp, start_state, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        a = rand(actions(mdp))
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end


function heuristic_policy(s)
    xtrue = s[1]
    ytrue = s[2]
    
    x = mod(s[1],20)
    y = mod(s[2],20)
    
    # If between bounds
    if (20 - x > 10) & (20 - x != 20) 
        a = :left
    elseif (20 - x <= 10) & (20 - x != 0)
        a = :right
    elseif 20 - y > 10
        a = :down
    else 
        a = :up
    end
    
#     # If outside of internal square 
#     if 60 - xtrue > 40 
#         a = :right
#     elseif 60 - xtrue < 20
#         a = :left 
#     elseif 60 - ytrue > 40
#         a = :up
#     elseif 60 - ytrue < 20
#         a = :down
#     else
#         return a
#     end
    
    return a 
end


function rollout_smart(mdp, start_state, heuristic_policy, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        a = heuristic_policy(s)
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end

# This code runs monte carlo simulations: you can calculate the mean and standard error from the results
results_random = [rollout(mdp, rand(initialstate(mdp))) for _ in 1:10000]
mean_random = mean(results_random)
SEM_smart = std(results_random)/(length(results_random)^0.5)
@show mean_random

results_smart = [rollout_smart(mdp, rand(initialstate(mdp)), heuristic_policy) for _ in 1:10000]
mean_smart = mean(results_smart)
SEM_smart = std(results_smart)/(length(results_smart)^0.5)
@show mean_smart

@show difference = mean_smart - mean_random


mean_random = -94.22497936090735
mean_smart = -18.60256773339322
difference = mean_smart - mean_random = 75.62241162751414


75.62241162751414

In [5]:
m = DenseGridWorld()

S = statetype(m)
A = actiontype(m)

# These would be appropriate containers for your Q, N, and t dictionaries:
n = Dict{Tuple{S, A}, Int}()
q = Dict{Tuple{S, A}, Float64}()
t = Dict{Tuple{S, A, S}, Int}()

# This is an example state - it is a StaticArrays.SVector{2, Int}
s = SA[19,19]
@show typeof(s)
@assert s isa statetype(m)

# here is an example of how to visualize a dummy tree (q, n, and t should actually be filled in your mcts code, but for this we fill it manually)
q[(SA[1,1], :right)] = 0.0
q[(SA[2,1], :right)] = 0.0
n[(SA[1,1], :right)] = 1
n[(SA[2,1], :right)] = 0
t[(SA[1,1], :right, SA[2,1])] = 1

q[[1,2],:right] = 1
q[s,:right] = 2



typeof(s) = StaticArraysCore.SVector{2, Int64}


2

In [6]:
############
# Question 3
############



# #struct MonteCarloTreeSearch
# 𝒫 # problem
# N # visit counts
# Q # action value estimates
# d # depth
# m # number of simulations
# c # exploration constant
# U # value function estimate
# end

##########################################
function bonus(N, Ns, s, a)
    if N(s,a) == 0
        return Inf
    else 
        return sqrt(log(Ns)/N[s,a])
    end    
end


##########################################

function explore(mdp,a,s,q,c)
    actions = actions(mdp)
    Ns = sum(n[s,a] for a in actions)
    
    values, indices = findmax(q[s,a] + c * bonus(n[s,a],Ns,s,a))
    return findmax(values)
end

##########################################
function simulateMCTS(s,a,n,q,d,c,V)
    if d <= 0 
        return V = 0
    end
    
    if !haskey(N, (s,first(a)) )
        for a in a
            n[s,a] = 0
            q[s,a] = 0
        end
        V = findmax(q(s,a) + 0.9*t(s,a))
    end
    a = explore(mdp,a,s,Q,c)
    sprime , r = @gen(:sp,:r)(mdp,s,a)
    q = r + 0.9 * simulateMCTS(sprime,a,n,Q,d-1,n,c,V)
    n[s,a] += 1
    return q[s,a] += (q - q([s,a]))/n[(s,a)]
    
end

        
##########################################
function MCTS(mdp, reps)
    s = [19,19]
    a = rand(actions(mdp))
    d = 0
    c = 0.8
    V = 0
    n = Dict{Tuple{S, A}, Int}()
    q = Dict{Tuple{S, A}, Float64}()
    t = Dict{Tuple{S, A, S}, Int}()
    
    
    
    for i in reps 
        q = simulateMCTS(s,a,n,q,d,c,V)
    end
    @show q
    values, idx = findmax(q[s,a]; dims = 2)[:]
        
    return findmax(values)

end


Q = MCTS(mdp,10)

inchrome(visualize_tree(q, n, t, SA[1,1]))

q = 0


LoadError: MethodError: no method matching getindex(::Int64, ::Vector{Int64}, ::Symbol)
[0mClosest candidates are:
[0m  getindex(::Number) at number.jl:95
[0m  getindex(::Union{AbstractChar, Number}, [91m::CartesianIndex{0}[39m) at multidimensional.jl:867
[0m  getindex(::Number, [91m::Integer[39m) at number.jl:96
[0m  ...

In [15]:
mdp = DenseGridWorld(seed = 4)

function bonus(N, Ns, s, a)
    if N(s,a) == 0
        return Inf
    else 
        return sqrt(log(Ns)/N[s,a])
    end    
end

function explore(mdp,a,s,q,n,c)
    actions = actions(mdp)
    Ns = sum(n[s,a] for a in actions)
    values, indices = findmax(q[s,a] + c * bonus(n,n[s,a],s,a))
    return actions[indices[rand(1:length(indices))]]
end

function simulateMCTS(mdp, s, a, q, n, t, d, c)
    if d <= 0
        return 0
    end
    
    if !haskey(n, (s,a))
        n[(s,a)] = 0
        q[(s,a)] = 0
        for sprime in states(mdp)
            t[(s,a,sprime)] = 0
        end
        V = simulateMCTS(mdp, s, a, q, n, t, d, c)
        q[(s,a)] = V
        return V
    end
    
    if n[(s,a)] == 0
        V = rollout(mdp, s, a)
        q[(s,a)] = V
        n[(s,a)] += 1
        return V
    end
    
    if all(n[(s,b)] > 0 for b in actions(mdp))
        b = explore(mdp, a, s, q, n, c)
        sprime, r = @gen(:sp,:r)(mdp,s,b)
        n[(s,b)] += 1
        t[(s,a,sprime)] += 1
        q[(s,a)] += (r + 0.9*simulateMCTS(mdp, sprime, b, q, n, t, d-1, c) - q[(s,a)]) / n[(s,a)]
        return q[(s,a)]
    else
        b = rand(actions(mdp))
        while n[(s,b)] > 0
            b = rand(actions(mdp))
        end
        sprime, r = @gen(:sp,:r)(mdp,s,b)
        n[(s,b)] += 1
        t[(s,a,sprime)] += 1
        q[(s,b)] = rollout(mdp, sprime, b)
        q[(s,a)] += (r + 0.9*q[(s,b)] - q[(s,a)]) / n[(s,a)]
        return q[(s,a)]
    end
end

function MCTS(mdp, reps)
    s = [19,19]
    a = rand(actions(mdp))
    d = 0
    c = 0.8
    V = 0
    n = Dict{Tuple{S, A}, Int}()
    q = Dict{Tuple{S, A}, Float64}()
    t = Dict{Tuple{S, A, S}, Int}()
    
    for i in 1:reps
        simulateMCTS(s,a,n,q,d,c,V)
    end
    
    Q = Dict{Tuple{S,A},Float64}()
    N = Dict{Tuple{S,A},Int}()
    
    for (sa, value) in q
        s, a = sa
        if haskey(n, sa)
            N[sa] = n[sa]
            if N[sa] > 0
                Q[sa] = value / N[sa]
            else
                Q[sa] = 0
            end
        else
            N[sa] = 0
            Q[sa] = 0
        end
    end
    
    for (sa, value) in n
        s, a = sa
        if !haskey(N, sa)
            N[sa] = value
        end
    end
    
    for s in states(mdp)
        for a in actions(mdp)
            if !haskey(t, (s,a,s))
                t[(s,a,s)] = 0
            end
        end
    end
    
    return Q, N, t
end

q , n , t = MCTS(mdp, 15)

inchrome(visualize_tree(q, n, t, SA[19,19] ))


LoadError: MethodError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

In [ ]:
############
# Question 4
############

# A starting point for the MCTS select_action function which can be used for Questions 3 and 4
function select_action(m, s)

    start = time_ns()
    n = Dict{Tuple{statetype(m), actiontype(m)}, Int}()
    q = Dict{Tuple{statetype(m), actiontype(m)}, Float64}()

    while time_ns() < start + 40_000_000 # run for a maximum of 40 ms to leave 10 ms to select an action
        break # replace this with mcts iterations to fill n and q
    end

    # select a good action based on q and/or n

    return rand(actions(m)) # this dummy function returns a random action, but you should return your selected action
end

############
# Question 4
############

HW3.evaluate(select_action, "your.gradescope.email@colorado.edu")

# If you want to see roughly what's in the evaluate function (with the timing code removed), check sanitized_evaluate.jl

########
# Extras
########

# With a typical consumer operating system like Windows, OSX, or Linux, it is nearly impossible to ensure that your function *always* returns within 50ms. Do not worry if you get a few warnings about time exceeded.

# You may wish to call select_action once or twice before submitting it to evaluate to make sure that all parts of the function are precompiled.

# Instead of submitting a select_action function, you can alternatively submit a POMDPs.Solver object that will get 50ms of time to run solve(solver, m) to produce a POMDPs.Policy object that will be used for planning for each grid world. You can achieve a score of 50 without doing this, but this may give you an advantage if you want to maximize your score.